# Modelo Condensación - Wikramanayake et al.
   

Implementa algunas fórmulas para describir la transferencia de calor en la condensación por gotas, teniendo en cuenta la distribución de tamaños de gotas, basado en los trabajos de Wrikamanayake (2019) y KimKim (2011) 

Se trata de un conjunto de resistencias térmicas que se deben vencer para la transferencia de calor.

In [8]:
## Importar librerías 

import matplotlib.pyplot as plt
import math
import numpy as np
import scipy
import scipy.integrate as integrate
from functools import partial


### Nomenclatura&nbsp;




**R_dif**=resistencia térmica por la capa de difusión

**T_amb**=Temperatura ambiente [K]

**T_int**=Temperatura de la gota [K]

**t_c**=Tasa de condensación por área. Hallada experimentalmente [Kg/m²s]

**h_fg**= Calor latente de condensación [J/kg]

**k_aire**=Conductividad térmica del K_aire [W/m K]

**delta**= grosor de la capa límite [m]

**S_r**=Área proyectada de la gota [m²]

**R_g**: Constante específica de los gases (8.3144598 / masa molar)    J/[Kg K]

**v_g**: Volumen especifico del vapor de agua (1/densidad vapor) [m²/Kg]

**h_fg**: Calor latente de vaporización [J/Kg]

**alfa**=Coeficiente de acomodación o condensación; fracción de moléculas de vapor que se mueven en la gota líquida durante el cambio de fase. Debe ser de entre 0.02 a 0.04 para agua. En Wrikamanayake et al. se usó 0.02

**T_s**=Temperatura de superficie de pared [K]

**T_sat**=Temperatura de saturación [K]

**k1**=Conductividad térmica del agua [W/mk]


**r**=radio de la gota [m]

**theta**=Ángulo de contacto [°]

**delta_coat**=Espesor de la capa recubrimiento [m]

**k_coat**=Conductividad térmica del recubrimiento [W/mK]

**rho**=Densidad del condensado [Kg/m³]

**k_c**=Conductividad térmica del condensado [W/mK]

**N_s**=número de sitios de nucleación en 10⁹ 1/m² 

**r_min**= radio de gota mínimo [m]

**r_max**= Radio de gota máximo [m]

**c**= Concentración vapor de agua en el aire [mol/m³]

**M_agua**=Peso molecular del agua [Kg/mol]

**D**=Coeficiente de difusión del vapor de agua en el aire [m²/s]

Se hacen las resistencias desde la parte exterior hasta el sustrato

### Parámetros de entrada

In [145]:
T_amb=296.1

T_sat=373
T_int=275.1
t_c= 2.7e-05 # Kg/m²s tomado del estado estable para 0V en wrikamaniyake et al
h_fg=2328*1000
k_aire=0.024
R_g=461.522 #Tomado del código de Sablowski
v_g=(1/0.5937643354220767)
alfa=0.02
T_s=275.1
theta=120
delta_coat=2e-06
k_coat=0.12 #CYTOP
k_c=0.58

k1=0.58
rho=997
N_s=2.5e9 # Tomado de KimKim
r_min=1.01669e-09 # [m]
r_max=0.0010561535753410094 # [m]

D=0.0000242 #m2/s
P_sat=0.024 #bar @21 C
P_atm=1 # bar
T_boil=373
M_agua=0.018015 #Kg/mol
c=0.68 #mol/m3
humedad=60 #%
R=8.31 # J/molK constante de los gases ideales

In [147]:
## Determinación grosor capa límite


D=22.5e-06 * (T_amb/273.15)**(1.8) ## [m²/s] Correlación de "Handbook of tables of applied engineering and science" de Boltz
x_i=(math.exp((-h_fg/R)*((1/T_int)-(1/T_boil))))
inf=humedad**(P_sat/P_atm)
delta= -D*M_agua*c*((math.exp((-h_fg/R)*((1/T_int)-(1/T_boil))))-(humedad**(P_sat/P_atm)))


print(delta)
print(D)

3.5161572698346664e-07
2.6016536671998145e-05


In [148]:

def R_dif_Wrikamanayake (r,T_amb, T_int, t_c, h_fg):
#----------------Resistencia a la difusión del vapor en la capa de no condensables [K/W] ----------------
#---- Se asume que el área superficial de la gota es lo mismo que el área proyectada

    R_dif=((T_amb-T_int)/(t_c*h_fg))/(2*math.pi*r**2)
    return R_dif

def R_cond_aire_Wrikamanayake (r,k_aire, delta):
#--------------Resistencia a la conducción del aire [K/W]---------------------
    R_cond_aire=delta/(k_aire*math.pi*(r**2))
    return R_cond_aire
#--Definición de h_i (coeficiente interfacial)
#h_i=Coeficiente interfacial de transferencia de Calor [W/m2K]
deltaT_sub=T_amb-T_s

h_i=((2*alfa)/(2-alfa))*(1/math.sqrt(2*math.pi*R_g*T_sat))*((h_fg**2)/(v_g*T_s))# Calcular de otra manera

def R_int_liq_Wrikamanayake (r, theta, alfa, T_s):


#--------Resistencia de Interfase líquido-gas [K/W]----------------------------

    R_int=1/(h_i*2*math.pi*(r**2)*(1-math.cos(theta)))

    return R_int

def R_cond_gotas_Wrikamanayake(r,theta, k1):
    #""""Resistencia a la conducción de las gotas [K/W]"""
   
    R_drop=theta/(4*math.pi*k1*r*math.sin(theta))

    return R_drop

def Q_total_n(r,R_dif2,T_amb,T_s):
 "Resistencia total por gotas pequeñas"""
 Q_total_n=((T_amb-T_s)/(((1/R_dif2(r)+(1/R_cond_dif(r)))**(-1))+R_int_liq(r)+R_got(r)))*n(r)
 return Q_total_n
    
def Q_total_N(r,R_dif2,T_amb,T_s,R_cond_dif,R_int_liq,R_got):
#Resistencia totala por gotas grandes'''
 Q_total_N = ((T_amb-T_s)/(((1/R_dif2(r)+(1/R_cond_dif(r)))**(-1))+R_int_liq(r)+R_got(r)))*N(r)
 return Q_total_N


def N_LeFevre(r, r_max):
 """ Distribución de tamaño de gotas para gotas grandes """
 N = 1/(3*math.pi*r**2*r_max) * (r/r_max)**(-2/3)
 return N

    
def r_e_KimKim(N_s):
 """ Radio efectivo de la gota """
 r_e = (4*N_s)**(-0.5)
 return r_e

r_e=r_e_KimKim(N_s)
    
def n_KimKim(r, deltaT_sub, r_min, delta_coat, k_coat, k_c, theta, h_i, rho, h_fg, r_e, r_max):
 """ Distribución de tamaño de gotas para gotas pequeñas """
 A_1 = deltaT_sub / (2*rho*h_fg)
 A_2 = theta * (1-math.cos(theta)) / (4*k_c*math.sin(theta))
 A_3 = 1/(2*h_i) + delta_coat*(1-math.cos(theta)) / (k_coat*(math.sin(theta))**2)
 tau = 3*r_e**2 * (A_2*r_e + A_3)**2 / (A_1*(11*A_2*r_e**2 - 14*A_2*r_e*r_min + 8*A_3*r_e - 11*A_3*r_min))
 B_1 = A_2/(tau*A_1) * ((r_e**2-r**2)/2 + r_min*(r_e-r) - r_min**2*math.log((r-r_min)/(r_e-r_min))) 
 B_2 = A_3/(tau*A_1) * (r_e-r - r_min*math.log((r-r_min)/(r_e-r_min)))
 n = 1/(3*math.pi*r_e**3*r_max) * (r_e/r_max)**(-2/3) * r*(r_e-r_min)/(r-r_min) * (A_2*r+A_3)/(A_2*r_e+A_3) * math.exp(B_1+B_2)
 return n 



# Definir funciones para la tasa de transferencia de calor

R_dif2 = partial(R_dif_Wrikamanayake,T_amb=T_amb, T_int=T_int, t_c=t_c, h_fg=h_fg)
R_dif2.___doc___="Resistencia a la difusión de no condensable"

R_cond_dif = partial(R_cond_aire_Wrikamanayake, k_aire=k_aire, delta=delta)
R_cond_dif.__doc__ = "Resistencia para conducción del aire  en cada gota dependiente del radio r en m"  
   
R_int_liq = partial(R_int_liq_Wrikamanayake, theta=theta, alfa=alfa, T_s=T_s)
R_int_liq.__doc__ = "Resistencia para interfase líquida dependiente del radio en m" 
    
    
R_got= partial(R_cond_gotas_Wrikamanayake,theta=theta, k1=k1)
R_got.__doc__ = "Resistencia conducción en las gotas, dependiente del r en m" 
    
    

n = partial(n_KimKim, deltaT_sub=deltaT_sub, r_min=r_min, delta_coat=delta_coat, k_coat=k_coat, k_c=k_c,
                theta=theta, h_i=h_i, rho=rho, h_fg=h_fg, r_e=r_e, r_max=r_max)
n.__doc__ = " Distribución de tamaño de gotas para tamaños pequeños dependiendo del radio r en m"
N = partial(N_LeFevre, r_max=r_max)
N.__doc__ = "drop size distribution for large drops depending on drop radius r in m"

   
    #Integración y cálculo del flujo de calor

"""Integración de los calores de acuerdo al número de gotas"""

    
q_n, q_n_interr = integrate.quad(Q_total_n, r_min, r_max,(R_dif2, T_amb, T_s))
q_N, q_N_interr = integrate.quad(Q_total_N, r_min, r_max,( R_dif2, T_amb, T_s, R_cond_dif, R_int_liq, R_got))
    
    
q = q_n + q_N

    
print ("r_e:",r_e,"\n"
      "q_N:",q_N,"W/m2\n"
      "q_n",q_n,"W/m2\n"
      "q:", round(q,2),"W/m2\n"
      "Relacion q_N/q:",round((q_N*100/q),2),"%\n"
       "x_i:",x_i,"\n"
      "inf",inf,"\n")

print ("h_i:", round(h_i,1), "W/m2k\n")




r_e: 1e-05 
q_N: 54893.09350481322 W/m2
q_n 1856.3062804955591 W/m2
q: 56749.4 W/m2
Relacion q_N/q: 96.73 %
x_i: 8.358003061478936e-117 
inf 1.1032543030428426 

h_i: 227218.5 W/m2k

